# ДЗ №4

1. Перенесите метрики в модуль src.metrics.py
2. Перенесите функцию prefilter_items в модуль src.utils.py
3. Создайте модуль src.recommenders.py. Напищите код для класса ниже 
(задание обсуждали на вебинаре, для первой функции практически сделали) и положите его в src.recommenders.py
4. Проверьте, что все модули корректно импортируются

In [355]:
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.dataset import load_data
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [356]:
train, test, product, baseline = load_data()

In [357]:
# найдем категории в которых есть продукты данного магазина
pcats = product[product['brand'] == 'Private'].groupby('department')['item_id'].nunique()
pcats = pcats[pcats>100]
pcats = product[product['department'].isin(pcats.index)]

In [358]:
n_items_before = train['item_id'].nunique()

data_train = prefilter_items(train, product_filter=pcats, top_n=5000)

n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 86865 to 5001


In [348]:
mr = MainRecommender(data_train)

In [359]:
mr.get_similar_users_recommendation(2500)

[5569471, 1058997, 862349, 1070820, 893501]

In [ ]:
baseline['mainrecommender'] = baseline['user_id'].apply(lambda x: mr.get_recommendations(x))

In [ ]:
baseline['similar_users'] = baseline['user_id'].apply(lambda x: mr.get_similar_users_recommendation(x))

In [ ]:
def compare_precision_at_5(row):
    flds = ['random_recommendation', 'popular_recommendation', 'itemitem', 'cosine', 
            'tfidf', 'own_purchases', 'mainrecommender', 'similar_users']
    for i in flds:
        row[f'precision_{i}'] = len(set(row[i]) & set(row['actual'])) / 5
    return row

precision = baseline.apply(lambda row: compare_precision_at_5(row), axis=1)
precision.describe().T